# Comparing different parameters for QAOA applied to MaxCut

Different parameters have been proposed for QAOA applied to MaxCut. Two notable examples are the "fixed angle conjecture" parameters of [1] and the parameters obtained for the SK model in the infinite size limit [2]. Both are available in QOKit. Below we show an example of comparing QAOA performance with the two parameter setting strategies.

[1] The fixed angle conjecture for QAOA on regular MaxCut graphs. Jonathan Wurtz, Danylo Lykov. https://arxiv.org/abs/2107.00677

[2] The Quantum Approximate Optimization Algorithm at High Depth for MaxCut on Large-Girth Regular Graphs and the Sherrington-Kirkpatrick Model. Joao Basso, Edward Farhi, Kunal Marwaha, Benjamin Villalonga, Leo Zhou. https://arxiv.org/abs/2110.14206

In [4]:
import numpy as np
import networkx as nx
from functools import partial

from qokit.qaoa_objective_maxcut import get_qaoa_maxcut_objective
from qokit.maxcut import maxcut_obj, get_adjacency_matrix
from qokit.utils import precompute_energies
from qokit.parameter_utils import get_fixed_gamma_beta, get_sk_gamma_beta
from qokit.qaoa_circuit_maxcut import get_parameterized_qaoa_circuit

In [2]:
import warnings
warnings.simplefilter("ignore", UserWarning)

In [3]:
# Generate a random regular graph
N = 20
d = 5
seed = 1
G = nx.random_regular_graph(d,N,seed=seed)

# precompute energies to speed up computation
obj = partial(maxcut_obj, w=get_adjacency_matrix(G))
precomputed_energies = precompute_energies(obj, N)
optimal_cut = np.max(precomputed_energies)

for p in range(1, 17):
    print(f"QAOA Approximation Ratio at p = {p}")
    f = get_qaoa_maxcut_objective(N, p, precomputed_cuts=precomputed_energies, parameterization='gamma beta')
    try:
        gamma, beta = get_fixed_gamma_beta(d, p)
        print(f'Fixed: {-f(gamma, beta) / optimal_cut}')
    except ValueError:
        # skip if parameters not available
        pass
    try:
        gamma, beta = get_sk_gamma_beta(p)
        print(f'SK: {-f(gamma / np.sqrt(d), beta) / optimal_cut}')
    except ValueError:
        # skip if parameters not available
        pass

QAOA Approximation Ratio at p = 1
Fixed: 0.7808446107491389
SK: 0.7810315788051738
QAOA Approximation Ratio at p = 2
Fixed: 0.8415702609691585
SK: 0.8427446925306544
QAOA Approximation Ratio at p = 3
Fixed: 0.8768876771219108
SK: 0.8755786913102327
QAOA Approximation Ratio at p = 4
Fixed: 0.9029594859012221
SK: 0.8992106234291608
QAOA Approximation Ratio at p = 5
SK: 0.9154819493745976
QAOA Approximation Ratio at p = 6
SK: 0.9270687431908406
QAOA Approximation Ratio at p = 7
SK: 0.9355272070467496
QAOA Approximation Ratio at p = 8
SK: 0.941317737448129
QAOA Approximation Ratio at p = 9
SK: 0.9461590286316792
QAOA Approximation Ratio at p = 10
SK: 0.9499925812865115
QAOA Approximation Ratio at p = 11
SK: 0.9529675051633442
QAOA Approximation Ratio at p = 12
SK: 0.9554558137531665
QAOA Approximation Ratio at p = 13
SK: 0.9575383013726461
QAOA Approximation Ratio at p = 14
SK: 0.9596475529911401
QAOA Approximation Ratio at p = 15
SK: 0.9616197361942117
QAOA Approximation Ratio at p = 16
S

In [8]:
get_parameterized_qaoa_circuit(G, 5)

In [10]:
qc.bind_parameters(np.hstack([angles['beta'], angles['gamma']]))

┌───┐                                                            »
 q2_0: ┤ H ├────────────────────────────────────────────────────────────»
       ├───┤                                                            »
 q2_1: ┤ H ├────────────────────────────────────────────────────────────»
       ├───┤                                                            »
 q2_2: ┤ H ├────────────────────────────────────────────────────────────»
       ├───┤                                                            »
 q2_3: ┤ H ├────────────────────────────────────────────────────────────»
       ├───┤                                                            »
 q2_4: ┤ H ├────────────────────────────────────────────────────────────»
       ├───┤                                                            »
 q2_5: ┤ H ├────────────────────────────────────────────────────────────»
       ├───┤                                                            »
 q2_6: ┤ H ├─■───────────────────────────────────────■──────────────────»
       ├───┤ │                                       │                  »
 q2_7: ┤ H ├─┼───────────────────■───────────────────┼──────────────────»
       ├───┤ │                   │                   │                  »
 q2_8: ┤ H ├─┼───────────────────┼───────────────────┼──────────────────»
       ├───┤ │                   │                   │                  »
 q2_9: ┤ H ├─┼───────────────────┼───────────────────┼──────────────────»
       ├───┤ │                   │                   │                  »
q2_10: ┤ H ├─┼───────────────────┼───────────────────┼──────────────────»
       ├───┤ │                   │                   │                  »
q2_11: ┤ H ├─┼───────────────────┼───────────────────┼──────────────────»
       ├───┤ │ZZ(-0.5*gamma[0])  │                   │                  »
q2_12: ┤ H ├─■───────────────────┼───────────────────┼──────────────────»
       ├───┤                     │                   │                  »
q2_13: ┤ H ├─────────────────────┼───────────────────┼──────────────────»
       ├───┤                     │                   │                  »
q2_14: ┤ H ├─────────────────────┼───────────────────┼──────────────────»
       ├───┤                     │                   │ZZ(-0.5*gamma[0]) »
q2_15: ┤ H ├─────────────────────┼───────────────────■──────────────────»
       ├───┤                     │                                      »
q2_16: ┤ H ├─────────────────────┼──────────────────────────────────────»
       ├───┤                     │ZZ(-0.5*gamma[0])                     »
q2_17: ┤ H ├─────────────────────■──────────────────────────────────────»
       ├───┤                                                            »
q2_18: ┤ H ├────────────────────────────────────────────────────────────»
       ├───┤                                                            »
q2_19: ┤ H ├────────────────────────────────────────────────────────────»
       └───┘                                                            »
«                                                                   »
« q2_0: ─────────────────────■──────────────────────────────────────»
«                            │                                      »
« q2_1: ─────────────────────┼──────────────────────────────────────»
«                            │                                      »
« q2_2: ─────────────────────┼───────────────────■──────────────────»
«                            │                   │                  »
« q2_3: ─────────────────────┼───────────────────┼──────────────────»
«                            │                   │                  »
« q2_4: ─■───────────────────┼───────────────────┼──────────────────»
«        │                   │                   │                  »
« q2_5: ─┼───────────────────┼───────────────────┼──────────────────»
«        │ZZ(-0.5*gamma[0])  │ZZ(-0.5*gamma[0])  │ZZ(-0.5*gamma[0]) »
« q2_6: ─■───────────────────■───────────────────■─────────────

array([-0.16418303,  0.04842084, -0.75725281, -0.01067492,  0.45568357])